In [1]:
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score

# Coleta dos Dados

In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [3]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

## Normalização das imagens

In [4]:
X_train = X_train / 255.0
X_test = X_test / 255.0

## Remodelagem (reshaping) a base de dados

In [5]:
# 6000 imagens, 28 de altura e 28 de largura
X_train.shape

(60000, 28, 28)

In [6]:
# altera de uma matriz para um vetor
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)
X_train.shape, X_test.shape

((60000, 784), (10000, 784))

In [7]:
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

In [8]:
lr = 0.01 
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = tf.keras.optimizers.SGD(lr=lr, decay= lr/comms_round, momentum=0.9)    

# Criação de Clientes

In [9]:
def create_clients(X, y, n_clients):
    indexes = np.arange(X.shape[0])
    np.random.shuffle(indexes)
    X = X[indexes]
    y = y[indexes]
    
    rate = X.shape[0]//n_clients
    X_slices = np.array([X[(i *  rate) : ((i + 1) * rate)] for i in range(n_clients)])
    y_slices = np.array([y[(i *  rate) : ((i + 1) * rate)] for i in range(n_clients)])
    return X_slices, y_slices

In [10]:
X_slices, y_slices = create_clients(X_train, y_train, 10)

In [11]:
X_slices.shape, y_slices.shape

((10, 6000, 784), (10, 6000, 10))

# Construção do Modelo

In [12]:
class MLP:
    def build(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784, )))
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
        return model

In [13]:
mlp = MLP()

global_model = mlp.build()

for j in range(comms_round):
    
    global_weights = global_model.get_weights()
    weights = []
    
    for i in range(10):
        client_model = mlp.build()
        client_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        client_model.set_weights(global_weights)
        X_slices, y_slices = create_clients(X_train, y_train, 10)
        
        client_model.fit(X_slices[i], y_slices[i], epochs=1, verbose=0)
        weights.append(client_model.get_weights())
    
    global_weights = (X_slices[i].shape[0] / X_train.shape[0]) * np.array(weights[0])
    for i in range(1,10):
        weights[i] = (X_slices[i].shape[0] / X_train.shape[0]) * np.array(weights[i])
        global_weights += np.array(weights[i])
    
    global_model.set_weights(global_weights)
    global_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
    
    test_loss, test_accuracy = global_model.evaluate(X_test, y_test)
    print("Round {}, Loss: {:.3f}, Accuracy: {:.3f}".format(j, test_loss, test_accuracy))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


313/313 [==============================] - 1s 1ms/step - loss: 0.6345 - accuracy: 0.7849
Round 0, Loss: 0.638, Accuracy: 0.782
313/313 [==============================] - 0s 1ms/step - loss: 0.5512 - accuracy: 0.8116
Round 1, Loss: 0.558, Accuracy: 0.807
313/313 [==============================] - 0s 1ms/step - loss: 0.5171 - accuracy: 0.8219
Round 2, Loss: 0.524, Accuracy: 0.817
313/313 [==============================] - 0s 1ms/step - loss: 0.4929 - accuracy: 0.8291
Round 3, Loss: 0.499, Accuracy: 0.824
313/313 [==============================] - 1s 1ms/step - loss: 0.4783 - accuracy: 0.8353
Round 4, Loss: 0.485, Accuracy: 0.828
313/313 [==============================] - 0s 999us/step - loss: 0.4660 - accuracy: 0.8370
Round 5, Loss: 0.474, Accuracy: 0.830
313/313 [==============================] - 0s 1ms/step - loss: 0.4584 - accuracy: 0.8428
Round 6, Loss: 0.465, Accuracy: 0.834
313/313 [==============================] - 0s 1ms/step - loss: 0.4498 - accuracy: 0.8447
Round 7, Loss: 0.457

313/313 [==============================] - 0s 1ms/step - loss: 0.3801 - accuracy: 0.8654
Round 64, Loss: 0.386, Accuracy: 0.863
313/313 [==============================] - 0s 1ms/step - loss: 0.3796 - accuracy: 0.8663
Round 65, Loss: 0.386, Accuracy: 0.862
313/313 [==============================] - 0s 1ms/step - loss: 0.3792 - accuracy: 0.8649
Round 66, Loss: 0.385, Accuracy: 0.862
313/313 [==============================] - 0s 1ms/step - loss: 0.3794 - accuracy: 0.8656
Round 67, Loss: 0.386, Accuracy: 0.861
313/313 [==============================] - 0s 1ms/step - loss: 0.3790 - accuracy: 0.8658
Round 68, Loss: 0.385, Accuracy: 0.863
313/313 [==============================] - 0s 997us/step - loss: 0.3795 - accuracy: 0.8657
Round 69, Loss: 0.385, Accuracy: 0.863
313/313 [==============================] - 0s 1ms/step - loss: 0.3783 - accuracy: 0.8665
Round 70, Loss: 0.384, Accuracy: 0.863
313/313 [==============================] - 0s 996us/step - loss: 0.3782 - accuracy: 0.8672
Round 71, L